# How to download and read the Solvency 2 legislation

In our first NLP project we will download, clean and read the Delegated Acts of the Solvency 2 legislation in all European languages.

In [1]:
import os
import re
import requests
import fitz

The languages of the European Union are
Bulgarian (BG),
Spanish (ES),
Czech (CS),
Danish (DA),
German (DE),
Estonian (ET),
Greek (EL),
English (EN),
French (FR),
Croatian (HR),
Italian (IT),
Latvian (LV),
Lithuanian (LT),
Hungarian (HU),
Maltese (MT),
Dutch (NL),
Polish (PL),
Portuguese (PT),
Romanian (RO),
Slovak (SK),
Solvenian (SL),
Finnish (FI),
Swedish (SV).

In [2]:
languages = ['BG','ES','CS','DA','DE','ET','EL',
             'EN','FR','HR','IT','LV','LT','HU',
             'MT','NL','PL','PT','RO','SK','SL',
             'FI','SV']

The urls of the Delegated Acts of Solvency 2 are constructed for these languages.

In [3]:
urls = ['https://eur-lex.europa.eu/legal-content/' + lang +
        '/TXT/PDF/?uri=OJ:L:2015:012:FULL&from=EN' 
        for lang in languages]

The following for loop retrieves the pdfs of the Delegated Acts from the website of the European Union and stores them in da_path.

In [4]:
da_path = '../../../../../10_central_data/legislation/'

for index in range(len(urls)):
    
    print("Retrieving " + languages[index] + ' from ' + urls[index])
    
    filename = 'Solvency II Delegated Acts - ' + languages[index]+ '.pdf'

    if not(os.path.isfile(da_path + filename)):
        
        r = requests.get(urls[index])

        f = open(da_path + filename,'wb+')
        f.write(r.content) 
        f.close()

        fh = open(da_path + filename, "rb")
        pdffile = PyPDF2.PdfFileReader(fh)
        fh.close()

Retrieving BG from https://eur-lex.europa.eu/legal-content/BG/TXT/PDF/?uri=OJ:L:2015:012:FULL&from=EN
Retrieving ES from https://eur-lex.europa.eu/legal-content/ES/TXT/PDF/?uri=OJ:L:2015:012:FULL&from=EN
Retrieving CS from https://eur-lex.europa.eu/legal-content/CS/TXT/PDF/?uri=OJ:L:2015:012:FULL&from=EN
Retrieving DA from https://eur-lex.europa.eu/legal-content/DA/TXT/PDF/?uri=OJ:L:2015:012:FULL&from=EN
Retrieving DE from https://eur-lex.europa.eu/legal-content/DE/TXT/PDF/?uri=OJ:L:2015:012:FULL&from=EN
Retrieving ET from https://eur-lex.europa.eu/legal-content/ET/TXT/PDF/?uri=OJ:L:2015:012:FULL&from=EN
Retrieving EL from https://eur-lex.europa.eu/legal-content/EL/TXT/PDF/?uri=OJ:L:2015:012:FULL&from=EN
Retrieving EN from https://eur-lex.europa.eu/legal-content/EN/TXT/PDF/?uri=OJ:L:2015:012:FULL&from=EN
Retrieving FR from https://eur-lex.europa.eu/legal-content/FR/TXT/PDF/?uri=OJ:L:2015:012:FULL&from=EN
Retrieving HR from https://eur-lex.europa.eu/legal-content/HR/TXT/PDF/?uri=OJ:L:20

# Data cleaning

If you look at the pdfs then you see that each page has a header with page number and information about the legislation and the language. These headers must be deleted to access the articles in the text.

In [5]:
DA_dict = dict({
                'BG': 'Официален вестник на Европейския съюз',
                'CS': 'Úřední věstník Evropské unie',
                'DA': 'Den Europæiske Unions Tidende',
                'DE': 'Amtsblatt der Europäischen Union',
                'EL': 'Επίσημη Εφημερίδα της Ευρωπαϊκής Ένωσης',
                'EN': 'Official Journal of the European Union',
                'ES': 'Diario Oficial de la Unión Europea',
                'ET': 'Euroopa Liidu Teataja',           
                'FI': 'Euroopan unionin virallinen lehti',
                'FR': "Journal officiel de l'Union européenne",
                'HR': 'Službeni list Europske unije',         
                'HU': 'Az Európai Unió Hivatalos Lapja',      
                'IT': "Gazzetta ufficiale dell'Unione europea",
                'LT': 'Europos Sąjungos oficialusis leidinys',
                'LV': 'Eiropas Savienības Oficiālais Vēstnesis',
                'MT': 'Il-Ġurnal Uffiċjali tal-Unjoni Ewropea',
                'NL': 'Publicatieblad van de Europese Unie',  
                'PL': 'Dziennik Urzędowy Unii Europejskiej',  
                'PT': 'Jornal Oficial da União Europeia',     
                'RO': 'Jurnalul Oficial al Uniunii Europene', 
                'SK': 'Úradný vestník Európskej únie',        
                'SL': 'Uradni list Evropske unije',            
                'SV': 'Europeiska unionens officiella tidning'})

The following code reads the pdfs, deletes the headers from all pages and saves the clean text to a .txt file.

In [6]:
DA = dict()

files = [f for f in os.listdir(da_path) if os.path.isfile(os.path.join(da_path, f))]    

print("Reading language ", end='')

for language in languages:

    print(language + " ", end='')

    if not("Delegated_Acts_" + language + ".txt" in files):
    
        # reading pages from pdf file
        da_pdf = fitz.open(da_path + 'Solvency II Delegated Acts - ' + language + '.pdf', 'rb')
        da_pages = [page.getText(output = "text") for page in da_pdf]
        da_pdf.close()

        # deleting page headers
        header = "17.1.2015\\s+L\\s+\\d+/\\d+\\s+" + DA_dict[language].replace(' ','\\s+') + "\\s+" + language + "\\s+"
        da_pages = [re.sub(header, '', page) for page in da_pages]
        DA[language] = ''.join(da_pages)
    
        # some preliminary cleaning -> should be more 
        DA[language] = DA[language].replace('\xad ', '')
    
        # saving txt file
        da_txt = open(da_path + "Delegated_Acts_" + language + ".txt", "wb")
        da_txt.write(DA[language].encode('utf-8'))
        da_txt.close()

    else:
        
        # loading txt file
        da_txt = open(da_path + "Delegated_Acts_" + language + ".txt", "rb")
        DA[language] = da_txt.read().decode('utf-8')
        da_txt.close()        

Reading language BG 

ValueError: bad type: 'stream'

# Retrieve the text within articles

Retrieving the text within articles is not straightforward. In English we have 'Article 1 some text', i.e. de word Article is put before the number. But some European languages put the word after the number and there are two languages, HU and LV, that put a dot between the number and the article. To  be able to read the text within the articles we need to know this ordering (and we need of course the word for article in every language).

In [ ]:
art_dict= dict({
                'BG': ['Член',      'pre'],
                'CS': ['Článek',    'pre'],
                'DA': ['Artikel',   'pre'],
                'DE': ['Artikel',   'pre'],
                'EL': ['Άρθρο',     'pre'],
                'EN': ['Article',   'pre'],
                'ES': ['Artículo',  'pre'],
                'ET': ['Artikkel',  'pre'],
                'FI': ['artikla',   'post'],
                'FR': ['Article',   'pre'],
                'HR': ['Članak',    'pre'],
                'HU': ['cikk',      'postdot'],
                'IT': ['Articolo',  'pre'],
                'LT': ['straipsnis','post'],
                'LV': ['pants',     'postdot'],
                'MT': ['Artikolu',  'pre'],
                'NL': ['Artikel',   'pre'],
                'PL': ['Artykuł',   'pre'],
                'PT': ['Artigo',    'pre'],
                'RO': ['Articolul', 'pre'],
                'SK': ['Článok',    'pre'],
                'SL': ['Člen',      'pre'],
                'SV': ['Artikel',   'pre']})

Next we can define a regex to select the text within an article.

In [ ]:
def retrieve_article(language, article_num):

    method = art_dict[language][1]
    
    if method == 'pre':
        string = art_dict[language][0] + ' ' + str(article_num) + '(.*?)' + art_dict[language][0] + ' ' + str(article_num + 1)
    elif method == 'post':
        string = str(article_num) + ' ' + art_dict[language][0] + '(.*?)' + str(article_num + 1) + ' ' + art_dict[language][0]
    elif method == 'postdot':
        string = str(article_num) + '. ' + art_dict[language][0] + '(.*?)' + str(article_num + 1) + '. ' + art_dict[language][0]

    r = re.compile(string, re.DOTALL)
            
    result = ' '.join(r.search(DA[language])[1].split())
            
    return result

Okay, where are we now? We have a function that can retrieve the text of all the articles in the Delegated Acts for each European language.

In [ ]:
retrieve_article('EN', 292)

In [ ]:
retrieve_article('DE', 292)

In [ ]:
retrieve_article('FR', 292)

In [ ]:
retrieve_article('EL', 292)

In [ ]:
retrieve_article('NL', 295)